In [ ]:
!rm -rf /root/.cache/kagglehub/datasets/hassanmojab/xview-dataset
print("🧹 Old dataset deleted")

🧹 Old dataset deleted


In [ ]:
!pip install -q kagglehub

import kagglehub

# Download the dataset
path = kagglehub.dataset_download("hassanmojab/xview-dataset")

print("✅ Downloaded to:", path)

100%|██████████| 19.4G/19.4G [03:07<00:00, 111MB/s]

Extracting files...


✅ Downloaded to: /root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1


In [ ]:
import os

path = "/root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1"
print("📂 Contents of downloaded dataset:")
print(os.listdir(path))

📂 Contents of downloaded dataset:
['__notebook_source__.ipynb', 'train_images', 'val_images', 'train_labels']


In [ ]:
pip cache purge

Files removed: 0


In [ ]:
!pip install -q ultralytics geopandas seaborn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.6 MB/s eta 0:00:00


In [ ]:
import os
import json
import shutil
import yaml
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image
from ultralytics import YOLO

In [ ]:
# ===============================
# 📂 DATASET PATHS
# ===============================
base_path = "/root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1"
train_img_dir = os.path.join(base_path, "train_images")
val_img_dir = os.path.join(base_path, "val_images")
label_file = os.path.join(base_path, "train_labels", "xView_train.geojson")

In [ ]:
label_file = os.path.join(base_path, "train_labels", "train_labels", "xView_train.geojson")

In [ ]:
# ===============================
# 🔠 CLASS MAPPING
# ===============================
xview_classes = {
    0: "Fixed-wing Aircraft", 1: "Small Aircraft", 2: "Cargo Plane", 3: "Helicopter",
    4: "Passenger Vehicle", 5: "Small Car", 6: "Bus", 7: "Pickup Truck",
    8: "Utility Truck", 9: "Truck", 10: "Cargo Truck", 11: "Truck w/Box",
    12: "Truck Tractor", 13: "Trailer", 14: "Truck w/Flatbed", 15: "Truck w/Liquid",
    16: "Crane Truck", 17: "Railway Vehicle", 18: "Passenger Car", 19: "Cargo Car",
    20: "Flat Car", 21: "Tank car", 22: "Locomotive", 23: "Container Ship",
    24: "Liner Ship", 25: "Fishing Vessel", 26: "Oil Tanker", 27: "Engineering Vehicle",
    28: "Tower crane", 29: "Container Crane", 30: "Reach Stacker", 31: "Straddle Carrier",
    32: "Mobile Crane", 33: "Dump Truck", 34: "Haul Truck", 35: "Scraper/Tractor",
    36: "Front Loader/Bulldozer", 37: "Excavator", 38: "Cement Mixer", 39: "Ground Grader",
    40: "Hut/Tent", 41: "Shed", 42: "Building", 43: "Aircraft Hangar", 44: "Damaged Building",
    45: "Facility", 46: "Construction Site", 47: "Vehicle Lot", 48: "Helipad",
    49: "Storage Tank", 50: "Shipping Container Lot", 51: "Shipping Container", 52: "Pylon",
    53: "Tower", 54: "Wind Turbine", 55: "Tower-like Structure", 56: "Container/Truck",
    57: "Oil Rig", 58: "Pier", 59: "Dock"
}

sorted_ids = sorted(xview_classes.keys())
typeid_to_yolo = {tid: i for i, tid in enumerate(sorted_ids)}
class_names = [xview_classes[tid] for tid in sorted_ids]

In [ ]:
# ===============================
# 📁 OUTPUT STRUCTURE
# ===============================
os.makedirs("images/train", exist_ok=True)
os.makedirs("images/val", exist_ok=True)

In [ ]:
os.makedirs("labels/train", exist_ok=True)
os.makedirs("labels/val", exist_ok=True)

In [ ]:
# # Copy images
# for fname in os.listdir(train_img_dir):
#     if fname.endswith(".tif"):
#         shutil.copy(os.path.join(train_img_dir, fname), os.path.join("images/train", fname))

# for fname in os.listdir(val_img_dir):
#     if fname.endswith(".tif"):
#         shutil.copy(os.path.join(val_img_dir, fname), os.path.join("images/val", fname))

In [ ]:
base_path = "/root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1"

train_img_src = os.path.join(base_path, "train_images", "train_images")
val_img_src   = os.path.join(base_path, "val_images", "val_images")

label_file = os.path.join(base_path, "train_labels", "xView_train.geojson")

In [ ]:
# ===============================
# 5️⃣ Convert GeoJSON to YOLO Labels
# ===============================
with open(label_file) as f:
    data = json.load(f)

created_labels = 0

for feat in data["features"]:
    props = feat["properties"]
    image_id = props.get("image_id")
    bbox = props.get("bounds_imcoords")
    type_id = props.get("type_id")

    if not image_id or not bbox or type_id not in typeid_to_yolo:
        continue

    try:
        x1, y1, x2, y2 = map(float, bbox.split(','))
    except:
        continue

    # Determine if the image is in train or val set
    if os.path.exists(os.path.join("images/train", image_id)):
        label_path = os.path.join("labels/train", os.path.splitext(image_id)[0] + ".txt")
        img_path = os.path.join("images/train", image_id)
    elif os.path.exists(os.path.join("images/val", image_id)):
        label_path = os.path.join("labels/val", os.path.splitext(image_id)[0] + ".txt")
        img_path = os.path.join("images/val", image_id)
    else:
        continue

    with Image.open(img_path) as img:
        img_w, img_h = img.size

    x_center = ((x1 + x2) / 2) / img_w
    y_center = ((y1 + y2) / 2) / img_h
    w = (x2 - x1) / img_w
    h = (y2 - y1) / img_h
    yolo_id = typeid_to_yolo[type_id]

    with open(label_path, "a") as out:
        out.write(f"{yolo_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")
        created_labels += 1

print(f"✅ Created {created_labels} YOLO-format labels")

✅ Created 265522 YOLO-format labels


In [ ]:
# ===============================
# 📄 CREATE data.yaml
# ===============================
data_yaml = {
    'train': 'images/train',
    'val': 'images/val',
    'nc': len(class_names),
    'names': class_names
}

with open("data.yaml", "w") as f:
    yaml.dump(data_yaml)

print("✅ data.yaml created!")

✅ data.yaml created!


In [ ]:
import yaml

xview_names = [
    "Fixed-wing Aircraft", "Small Aircraft", "Cargo Plane", "Helicopter", "Passenger Vehicle", "Small Car", "Bus",
    "Pickup Truck", "Utility Truck", "Truck", "Cargo Truck", "Truck w/Box", "Truck Tractor", "Trailer",
    "Truck w/Flatbed", "Truck w/Liquid", "Crane Truck", "Railway Vehicle", "Passenger Car", "Cargo Car",
    "Flat Car", "Tank car", "Locomotive", "Container Ship", "Liner Ship", "Fishing Vessel", "Oil Tanker",
    "Engineering Vehicle", "Tower crane", "Container Crane", "Reach Stacker", "Straddle Carrier", "Mobile Crane",
    "Dump Truck", "Haul Truck", "Scraper/Tractor", "Front Loader/Bulldozer", "Excavator", "Cement Mixer",
    "Ground Grader", "Hut/Tent", "Shed", "Building", "Aircraft Hangar", "Damaged Building", "Facility",
    "Construction Site", "Vehicle Lot", "Helipad", "Storage Tank", "Shipping Container Lot", "Shipping Container",
    "Pylon", "Tower", "Wind Turbine", "Tower-like Structure", "Container/Truck", "Oil Rig", "Pier", "Dock"
]

data_yaml = {
    'train': 'images/train',
    'val': 'images/val',
    'nc': 60,
    'names': xview_names
}

# Overwrite and save cleanly
with open("data.yaml", "w", encoding='utf-8') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print("✅ data.yaml written successfully")

# Let's also double-check:
with open("data.yaml", "r") as f:
    print("\n📄 data.yaml preview:")
    print(f.read())


✅ data.yaml written successfully

📄 data.yaml preview:
names:
- Fixed-wing Aircraft
- Small Aircraft
- Cargo Plane
- Helicopter
- Passenger Vehicle
- Small Car
- Bus
- Pickup Truck
- Utility Truck
- Truck
- Cargo Truck
- Truck w/Box
- Truck Tractor
- Trailer
- Truck w/Flatbed
- Truck w/Liquid
- Crane Truck
- Railway Vehicle
- Passenger Car
- Cargo Car
- Flat Car
- Tank car
- Locomotive
- Container Ship
- Liner Ship
- Fishing Vessel
- Oil Tanker
- Engineering Vehicle
- Tower crane
- Container Crane
- Reach Stacker
- Straddle Carrier
- Mobile Crane
- Dump Truck
- Haul Truck
- Scraper/Tractor
- Front Loader/Bulldozer
- Excavator
- Cement Mixer
- Ground Grader
- Hut/Tent
- Shed
- Building
- Aircraft Hangar
- Damaged Building
- Facility
- Construction Site
- Vehicle Lot
- Helipad
- Storage Tank
- Shipping Container Lot
- Shipping Container
- Pylon
- Tower
- Wind Turbine
- Tower-like Structure
- Container/Truck
- Oil Rig
- Pier
- Dock
nc: 60
train: images/train
val: images/val



In [ ]:
# ===============================
# 🧠 TRAIN YOLOv8
# ===============================
model = YOLO("yolov8m.pt")  # Use 'n', 's', 'l', or 'x' as needed

model.train(
    data="data.yaml",
    epochs=100,
    imgsz=640,
    patience=10,
    batch=16,
    workers=2,
    name="xview_yolo8",
    augment=True,
    pretrained=True,
    verbose=True
)

Ultralytics 8.3.144 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=xview_yolo83, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=True, profil

train: Scanning /content/labels/train... 0 images, 846 backgrounds, 0 corrupt: 100%|██████████| 846/846 [00:07<00:00, 114.31it/s]

WARNING ⚠️ train: No labels found in /content/labels/train.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
train: New cache created: /content/labels/train.cache
WARNING ⚠️ Labels are missing or empty in /content/labels/train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 397.6±536.5 MB/s, size: 24647.1 KB)


val: Scanning /content/labels/val... 0 images, 281 backgrounds, 0 corrupt: 100%|██████████| 281/281 [00:10<00:00, 28.02it/s]

WARNING ⚠️ val: No labels found in /content/labels/val.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: New cache created: /content/labels/val.cache
WARNING ⚠️ Labels are missing or empty in /content/labels/val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


Plotting labels to runs/detect/xview_yolo83/labels.jpg... 
WARNING ⚠️ zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000156, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/xview_yolo83
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100       6.2G          0      240.3          0          0        640:  64%|██████▍   | 34/53 [01:35<00:53,  2.81s/it]


KeyboardInterrupt: 

In [ ]:
import os
import shutil

# Update base path if needed
base_path = "/root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1"
train_img_src = os.path.join(base_path, "train_images", "train_images")
val_img_src = os.path.join(base_path, "val_images", "val_images")

train_img_dst = "images/train"
val_img_dst = "images/val"

# Copy .tif images from original dataset into YOLO folders
def copy_images(src, dst):
    copied = 0
    for file in os.listdir(src):
        if file.endswith(".tif"):
            shutil.copy(os.path.join(src, file), os.path.join(dst, file))
            copied += 1
    print(f"✅ Copied {copied} images to {dst}")

copy_images(train_img_src, train_img_dst)
copy_images(val_img_src, val_img_dst)

✅ Copied 846 images to images/train
✅ Copied 281 images to images/val


In [ ]:
import os

base_path = "/root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1"

print("📁 train_images:")
train_list = os.listdir(os.path.join(base_path, "train_images"))
print(train_list[:10], f"... total: {len(train_list)} files")

print("\n📁 val_images:")
val_list = os.listdir(os.path.join(base_path, "val_images"))
print(val_list[:10], f"... total: {len(val_list)} files")


📁 train_images:
['train_images'] ... total: 1 files

📁 val_images:
['val_images'] ... total: 1 files


In [ ]:
import os

label_base = "/root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1/train_labels"

print("📂 Listing contents of val_labels folder:")
for root, dirs, files in os.walk(label_base):
    print(f"\n📁 Directory: {root}")
    for file in files:
        print(" └──", file)

📂 Listing contents of train_labels folder:

📁 Directory: /root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1/train_labels
 └── xView_train.geojson


In [ ]:
import os
val_images = os.listdir("images/val")
print("🖼️ Sample val image filenames:", val_images[:5])


🖼️ Sample val image filenames: ['2132.tif', '1833.tif', '357.tif', '2415.tif', '1935.tif']


In [ ]:
import json

with open(label_file) as f:
    data = json.load(f)

geojson_image_ids = set(f["properties"]["image_id"] for f in data["features"])
missing_labels = [img for img in val_images if img not in geojson_image_ids]

print(f"🧩 {len(missing_labels)} val images are missing in the geojson.")
print("Example:", missing_labels[:5])


🧩 281 val images are missing in the geojson.
Example: ['2132.tif', '1833.tif', '357.tif', '2415.tif', '1935.tif']


In [ ]:
import os

label_base = "/root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1/val_labels"

print("📂 Listing contents of val_labels folder:")
for root, dirs, files in os.walk(label_base):
    print(f"\n📁 Directory: {root}")
    for file in files:
        print(" └──", file)

📂 Listing contents of val_labels folder:


In [ ]:
import os, shutil, json, random
from PIL import Image
import yaml

# === SETUP PATHS ===
base_path = "/root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1"
image_source = os.path.join(base_path, "train_images", "train_images")
geojson_path = os.path.join(base_path, "train_labels", "xView_train.geojson")

# === CLEAN FOLDERS ===
for f in ["images/train", "images/val", "labels/train", "labels/val"]:
    shutil.rmtree(f, ignore_errors=True)
    os.makedirs(f, exist_ok=True)

# === CLASS MAPPING ===
xview_classes = {
    0: "Fixed-wing Aircraft", 1: "Small Aircraft", 2: "Cargo Plane", 3: "Helicopter",
    4: "Passenger Vehicle", 5: "Small Car", 6: "Bus", 7: "Pickup Truck", 8: "Utility Truck",
    9: "Truck", 10: "Cargo Truck", 11: "Truck w/Box", 12: "Truck Tractor", 13: "Trailer",
    14: "Truck w/Flatbed", 15: "Truck w/Liquid", 16: "Crane Truck", 17: "Railway Vehicle",
    18: "Passenger Car", 19: "Cargo Car", 20: "Flat Car", 21: "Tank car", 22: "Locomotive",
    23: "Container Ship", 24: "Liner Ship", 25: "Fishing Vessel", 26: "Oil Tanker",
    27: "Engineering Vehicle", 28: "Tower crane", 29: "Container Crane", 30: "Reach Stacker",
    31: "Straddle Carrier", 32: "Mobile Crane", 33: "Dump Truck", 34: "Haul Truck",
    35: "Scraper/Tractor", 36: "Front Loader/Bulldozer", 37: "Excavator", 38: "Cement Mixer",
    39: "Ground Grader", 40: "Hut/Tent", 41: "Shed", 42: "Building", 43: "Aircraft Hangar",
    44: "Damaged Building", 45: "Facility", 46: "Construction Site", 47: "Vehicle Lot",
    48: "Helipad", 49: "Storage Tank", 50: "Shipping Container Lot", 51: "Shipping Container",
    52: "Pylon", 53: "Tower", 54: "Wind Turbine", 55: "Tower-like Structure", 56: "Container/Truck",
    57: "Oil Rig", 58: "Pier", 59: "Dock"
}
sorted_ids = sorted(xview_classes.keys())
typeid_to_yolo = {tid: i for i, tid in enumerate(sorted_ids)}

# === PARSE GEOJSON ===
with open(geojson_path) as f:
    data = json.load(f)

annotations = {}
for feat in data["features"]:
    props = feat["properties"]
    image_id = props.get("image_id")
    bbox = props.get("bounds_imcoords")
    cls = props.get("type_id")

    if not image_id or not bbox or cls not in typeid_to_yolo:
        continue

    try:
        x1, y1, x2, y2 = map(float, bbox.split(","))
    except:
        continue

    annotations.setdefault(image_id, []).append((typeid_to_yolo[cls], x1, y1, x2, y2))

# === SPLIT 80/20 ===
image_ids = list(annotations.keys())
random.shuffle(image_ids)
split = int(len(image_ids) * 0.8)
train_ids, val_ids = image_ids[:split], image_ids[split:]

# === COPY IMAGES + CONVERT LABELS ===
def process_set(image_ids, img_dst, lbl_dst):
    count = 0
    for img_name in image_ids:
        src = os.path.join(image_source, img_name)
        dst = os.path.join(img_dst, img_name)
        if not os.path.exists(src):
            continue
        try:
            with Image.open(src) as im:
                w, h = im.size
        except:
            continue
        shutil.copy(src, dst)

        label_file = os.path.splitext(img_name)[0] + ".txt"
        label_path = os.path.join(lbl_dst, label_file)

        with open(label_path, "w") as f:
            for cls, x1, y1, x2, y2 in annotations[img_name]:
                xc = ((x1 + x2) / 2) / w
                yc = ((y1 + y2) / 2) / h
                bw = (x2 - x1) / w
                bh = (y2 - y1) / h
                f.write(f"{cls} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")
        count += 1
    return count

train_count = process_set(train_ids, "images/train", "labels/train")
val_count   = process_set(val_ids,   "images/val",   "labels/val")

print(f"✅ {train_count} training images and {val_count} validation images processed")

# === WRITE data.yaml ===
data_yaml = {
    "train": "images/train",
    "val": "images/val",
    "nc": len(xview_classes),
    "names": [xview_classes[i] for i in sorted_ids]
}
with open("data.yaml", "w") as f:
    yaml.dump(data_yaml, f)

print("✅ data.yaml written — ready to train!")


✅ 614 training images and 154 validation images processed
✅ data.yaml written — ready to train!


In [ ]:
model = YOLO("yolov8m.pt")  # Use 'n', 's', 'l', or 'x' as needed

model.train(
    data="data.yaml",
    epochs=15,
    imgsz=1024,
    patience=3,
    batch=16,
    workers=2,
    name="xview_yolo8",
    augment=True,
    pretrained=True,
    verbose=True
)

Ultralytics 8.3.144 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=xview_yolo84, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile

train: Scanning /content/labels/train.cache... 614 images, 0 backgrounds, 172 corrupt: 100%|██████████| 614/614 [00:00<?, ?it/s]

train: /content/images/train/100.tif: ignoring corrupt image/label: negative label values [  -0.001924]
train: /content/images/train/1036.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0003      1.0003]
train: /content/images/train/1037.tif: ignoring corrupt image/label: negative label values [  -0.001095   -0.000156   -0.000782]
train: /content/images/train/1050.tif: ignoring corrupt image/label: negative label values [    -0.0014]
train: /content/images/train/1051.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0012]
train: /content/images/train/1063.tif: ignoring corrupt image/label: negative label values [  -0.003622]
train: /content/images/train/1076.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0003]
train: /content/images/train/1080.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0055       1.007      1.0052]
train: /conte

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 151.2±52.0 MB/s, size: 28874.2 KB)


val: Scanning /content/labels/val.cache... 154 images, 0 backgrounds, 44 corrupt: 100%|██████████| 154/154 [00:00<?, ?it/s]

val: /content/images/val/1046.tif: ignoring corrupt image/label: negative label values [  -0.001095]
val: /content/images/val/107.tif: ignoring corrupt image/label: negative label values [  -0.065401]
val: /content/images/val/1090.tif: ignoring corrupt image/label: negative label values [  -0.000156]
val: /content/images/val/1104.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0114      1.0014]
val: /content/images/val/1109.tif: ignoring corrupt image/label: negative label values [   -0.00189]
val: /content/images/val/1121.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0008]
val: /content/images/val/1158.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0161      1.0079      1.0042]
val: /content/images/val/1178.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0042]
val: /content/images/val/1181.tif: ignoring corrupt image/label: nega

Plotting labels to runs/detect/xview_yolo84/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000156, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 1024 train, 1024 val
Using 2 dataloader workers
Logging results to runs/detect/xview_yolo84
Starting training for 15 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/28 [00:00<?, ?it/s]

WARNING ⚠️ CUDA OutOfMemoryError in TaskAlignedAssigner, using CPU


In [ ]:
model.train(
    data='data.yaml',
    epochs=100,
    imgsz=1024,
    patience=10,
    batch=8,
    workers=8,
    name='xview_yolo8_tuned',
    augment=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=45,
    flipud=0.5,
    mosaic=1.0,
    mixup=0.2,
    pretrained=True,
    verbose=True,
    lr0=0.001,      # initial learning rate
    lrf=0.01,       # final learning rate (as a fraction of lr0)
    optimizer='AdamW',
    cos_lr=True     # use cosine annealing schedule
)